本章以第 9 章定义的二维向量 Vector2d 类为基础，向前迈出一大步，定义表示多维向量的 Vector 类。这个类的行为与 Python 标准中的不可变扁平序列一样。Vector 实例中的元素是浮点数，本章结束后 Vector2d 类将支持以下功能

- 基本的序列协议 -- `__len__` 和 `__getitem__`
- 正确表述拥有很多元素的实例
- 适当的切片支持，用于生成新的 Vector 实例
- 综合各个元素的值计算散列值
- 自定义的格式语言扩展

此外，我们还将通过 `__getattr__` 方法实现属性的动态存取，以此取代 Vector2d 使用的只读属性 -- 不过，序列类型通常不会这么做

在大量代码之间，我们将穿插讨论一个概念：把协议当做正式借口。我们将说明协议和鸭子类型之间的关系，以及对自定义类型的影响

## Vector 第一版：与 Vector2d 类兼容

Vector 类要尽量与上一章的 Vector2d 类兼容。为了编写 Vector(3, 4)，Vector(3, 4, 5) 这样的代码，我们可以让 `__init__` 方法接受任意个参数（通过 `*args`)；但是，序列类型的构造方法最好接受可迭代的对象为参数，因为所有内置的序列类型都是这样做的。下面是我们的第一版 Vector 代码

In [8]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components) # 把 Vector 分量保存到一个数组中('d' 表示 double)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        # 使用 reprlib.repr() 函数获取 self._commponents 有限长度表示，如 array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] # 将字符串插入构造方法调用之前，去掉前面的 'd' 和后面的 )
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(self._components))     
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self)) # 迭代计算向量长度
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) #我们只需要改动一行，直接把 memoryview 传给构造方法，不用使用 * 拆包

In [9]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [10]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [11]:
Vector(range(10)) # reprlib.repr 限制了长度

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

我们使用 reprlib.repr 的方式需要做些说明。这个函数用于生成大型结构或递归结构的安全表示形式，它会限制输出字符串的长度，用 '...' 表示截断的部分。另外我们希望 Vector 实例的表现形式是 Vector([3.0, 4.0, 5.0])，而不是 Vector(array('d', [3.0, 4.0, 5.0]))，因为 Vector 实例中的数组是实现细节。因为这两种构造方法调用方式所构建的 Vector 对象是一样的，所以我选择使用更简单的语法，即传入列表参数

编写 `__repr__` 方法时，本可以使用这个表达生成简化的 components 显示形式：`reprlib.repr(list(self._components))`，然而，这么做有些浪费，因为要把 `self._components` 中的每一个元素复制到一个列表中，然后使用列表的表现形式。我没有这么做，而是直接把 `self._components` 传给 `reprlib.repr` 函数，然后去掉 [ ] 外面的字符。

> 调用 repr() 函数的目的是调试，因此绝对不能抛出异常，如果 `__repr__` 方法实现有问题，那么必须处理，尽量输出有用的内容，让用户能够识别目标对象

注意，`__str__`, `__eq__` 和 `__bool__` 方法与 Vector2d 类中的一样，而 frombytes 方法也只是把 `*` 去掉了。这是 Vector2d 可迭代的好处之一

顺便说一下，我们本可以让 Vector 继承 Vector2d，但是没有这么做，原因有两点，一是两个构造方法不兼容，所以不建议继承，这一点可以通过适当处理 `__init__` 方法解决，第二个原因更重要：我想把 Vector 类作为单独的示例，因此实现序列协议，接下来我们讨论 协议 这个术语，然后实现序列协议

## 协议和鸭子类型

在第一章我们就说过，Python 中创建功能完善的序列类型无需使用继承，只需实现符合序列协议的方法

在面向对象中，协议是非正式的接口，只在文档中定义，在代码中不定义。例如，Python 的序列协议只需要 `__len__` 和 `__getitem__` 两个方法。任何类（如 Spam）只要使用标准签名和语义实现了这两个方法，就能用在任何期待序列的地方。Spam 是不是哪个类的子类无关紧要，只要提供了所需方法即可。第一章见过一个例子，下面再次给出代码：

In [12]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamons clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

FrenchDeck 类能充分利用 Python 的很多功能，因为它实现了序列协议，不过代码中并没有声明这一点。任何有经验的 Python 程序员只要看一眼就知道它是序列，即便它是 Object 的子类也无妨。我们说它是序列，因为它的行为像序列，这才是重点

Alex Martelli 说不要检查它是不是鸭子，而是看它的叫声像不像鸭子，走路姿势像不像鸭子，等等，这样的类人称鸭子类型

协议是非正式的，没有强制力，因此如果你知道类的具体使用场景，通常只需要实现一个协议的部分。例如，为了支持迭代，只需要实现 `__getitem__` 方法，没必要提供 `__len__` 方法。

## Vector 类第 2 版：可切片序列